In [1]:
using Pkg
Pkg.activate("C:\\Users\\Alex\\Desktop\\repo\\2612\\0903\\dynamical-systems\\env\\integrate\\")
using StaticArrays, DifferentialEquations, DynamicalSystems, BenchmarkTools

  Activating project at `C:\Users\Alex\Desktop\repo\2612\0903\dynamical-systems\env\integrate`


In [30]:
# u[1] - E; u[2] - x; u[3] - y
# α - 1, τ - 2, τD - 3, τy - 4, J - 5, xthr - 6, ythr - 7, U0 - 8, ΔU0 - 9, β - 10, I0 -11

In [11]:
@inbounds U(y, p) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
@inbounds σ(x, p) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )
@inbounds g(E, x, y, p, U_) = log( 1.0 + exp( (p[5] * U_ * x * E + p[11]  ) / (p[1]) ) )

@inbounds function TM(u, p, t)
    
    U_ = U(u[3], p)
    
    du1 = (-u[1] + p[1] * g(u[1], u[2], u[3], p, U_) ) / p[2]
    du2 = (1.0 - u[2]) / p[3] - U_*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2], p)
    
    return SVector(du1, du2, du3)
end;

In [18]:
t = 5000; tt = 1000; tstep = 0.001
trange = range(0.0, t+tt, step = tstep)
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [19]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [20]:
u0 = SA[17.6055, 0.6181, 0.429232]
p  = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)

3-dimensional CoupledODEs
 deterministic: true
 discrete time: false
 in-place:      false
 dynamic rule:  TM
 ODE solver:    RK4
 ODE kwargs:    (adaptive = false, dt = 0.001)
 parameters:    [1.58, 0.013, 0.08, 3.3, 3.07, 0.75, 0.4, 0.3, 0.305, 0.3, -1.4]
 time:          0.0
 state:         [17.6055, 0.6181, 0.429232]


In [21]:
@benchmark trajectory(ds, t, Δt = tstep; Ttr = tt)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.997 s …   2.016 s  ┊ GC (min … max): 0.00% … 0.04%
 Time  (median):     1.999 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.004 s ± 10.227 ms  ┊ GC (mean ± σ):  0.01% ± 0.02%

  █      █                                                █  
  █▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2 s            Histogram: frequency by time        2.02 s <

 Memory estimate: 114.44 MiB, allocs estimate: 14.